In [22]:

import time
import requests
import pandas as pd

## gender  : F or M
## code : 2 for 10k, 4 for half

import time
import requests
import pandas as pd

def get_race_records(gender="M", year=2023, run_type="2"):
    headers = {
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'accept-language': 'en-US,en;q=0.5',
        'content-type': 'application/json',
        'origin': 'https://www.seoulhalfmarathon.com',
        'referer': 'https://www.seoulhalfmarathon.com/',
        'sec-ch-ua': '"Brave";v="123", "Not:A-Brand";v="8", "Chromium";v="123"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site',
        'sec-gpc': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    }

    json_data = {
        'type': 'year',
        'code': run_type,
        'year': str(year),
        'gender': gender,
        'page': 0,
    }

    all_records = []

    while True:
        response = requests.post('https://marathonapi.chosun.com/v1/half-record/history', headers=headers, json=json_data)
        if response.status_code != 200:  # Check if the HTTP request was successful
            print(f"Failed to fetch data: {response.status_code} - {response.text}")
            break

        result = response.json()
        print(result)  # For debugging purposes, to see the API response structure

        if 'data' not in result or not result['data'] or 'halfRecordDtoList' not in result['data']:
            print("No more data or wrong data format received.")
            break

        records = result['data']['halfRecordDtoList']
        if not records:
            print("No records found on the current page.")
            break

        all_records.extend(records)
        json_data['page'] += 1
        time.sleep(0.4)  # Respect the API's rate limits

    return all_records


In [23]:
print(get_race_records("M",2021,"2"))

Failed to fetch data: 404 - {"status":404,"message":"NOT_FOUND","data":"조회 결과가 없습니다."}
[]


In [24]:
years = [2018,2019,2020,2021,2022,2023]

result_for_multiple_years = []
for year in years:
  print(year)
  year_result = get_race_records("M",year,"2")
  result_for_multiple_years.extend(year_result)
  

2018
{'status': 200, 'message': 'OK', 'data': {'totalCnt': 2412, 'halfRecordDtoList': [{'no': 24533, 'year': '2018', 'code': '2', 'baebun': '11361', 'gender': 'M', 'age': None, 'rec5km': '0:17:40', 'rec10km': '', 'rec15km': '', 'rec20km': '', 'netFull': '0:35:27', 'gunFull': '0:35:31', 'rankGender': '1', 'rankTotal': 1, 'recPace': '3:33'}, {'no': 24534, 'year': '2018', 'code': '2', 'baebun': '12432', 'gender': 'M', 'age': None, 'rec5km': '0:17:40', 'rec10km': '', 'rec15km': '', 'rec20km': '', 'netFull': '0:36:05', 'gunFull': '0:36:08', 'rankGender': '2', 'rankTotal': 2, 'recPace': '3:37'}, {'no': 24535, 'year': '2018', 'code': '2', 'baebun': '12338', 'gender': 'M', 'age': None, 'rec5km': '0:17:50', 'rec10km': '', 'rec15km': '', 'rec20km': '', 'netFull': '0:36:51', 'gunFull': '0:36:55', 'rankGender': '3', 'rankTotal': 3, 'recPace': '3:42'}, {'no': 24536, 'year': '2018', 'code': '2', 'baebun': '12930', 'gender': 'M', 'age': None, 'rec5km': '0:17:50', 'rec10km': '', 'rec15km': '', 'rec20k

In [5]:
records = get_race_records(gender="M", year=2023)


In [25]:

df = pd.DataFrame(result_for_multiple_years)
df.to_excel("10km_seoul_records.xlsx")

In [26]:
# Convert time strings to timedelta objects
time_columns = ['rec5km', 'rec10km', 'rec15km', 'rec20km', 'netFull', 'gunFull']  # Adjust this list as needed
for col in time_columns:
    df[col] = pd.to_timedelta(df[col].fillna('0:00:00'))

# Calculate the median time for each year for the specific time column you're interested in
# Here's an example calculating the median of 'netFull' times
median_times = df.groupby('year')['netFull'].median()

# Convert median times back to strings if needed
median_times_str = median_times.dt.components.apply(lambda x: f"{int(x['hours']):02d}:{int(x['minutes']):02d}:{int(x['seconds']):02d}", axis=1)

print(median_times_str)

year
2018    01:01:09
2019    01:00:18
2023    00:56:57
dtype: object
